In [18]:
from MLForecastPipeline import *

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

LAG_TRANSFORMS_MAP = {
    "expanding_mean_rolling_14_rolling_30": {1: 'expanding_mean', 7: 'expanding_mean', 30: 'rolling_mean_30'},
    "expanding_mean_rolling_14": {1: 'expanding_mean', 7: 'rolling_mean_14', 30: 'expanding_mean'},
    "rolling_14_rolling_30_expanding": {1: 'rolling_mean_14', 7: 'rolling_mean_30', 30: 'expanding_mean'},
    "rolling_14_expanding": {1: 'rolling_mean_14', 30: 'expanding_mean'},
    "rolling_14_only": {1: 'rolling_mean_14'},
    "no_transform": {},
}

def map_lag_transforms(lag_transform_dict, lag_transforms_map=LAG_TRANSFORMS_MAP):
    for name, transform in lag_transforms_map.items():
        if lag_transform_dict == transform:
            return name
    return "unknown"

def analyze_results(df, lag_transforms_map=LAG_TRANSFORMS_MAP, mape_threshold=40, model_filter=None):
    df = df.copy()
    df['Lag Transform Name'] = df['Lag Transforms'].apply(lambda x: map_lag_transforms(x, lag_transforms_map))
    df['Lag_Set_Name'] = df['Lag Name']
    # Identify MAPE columns dynamically
    mape_columns = [col for col in df.columns if col.startswith("test_") and col.endswith("_days")]
    
    # Compute mean MAPE across all test periods
    df['MAPE'] = df[mape_columns].mean(axis=1)
    
    # Apply filtering
    top_df = df[df["MAPE"] < mape_threshold].copy()
    if model_filter:
        top_df = top_df[top_df['Model'] == model_filter].copy()
    
    # Compute groupings
    top_models = top_df.groupby("Model")["MAPE"].mean().sort_values().reset_index()
    top_transforms = top_df.groupby("Transforms")["MAPE"].mean().sort_values().reset_index()
    top_lag_transforms = top_df.groupby("Lag Transform Name")["MAPE"].mean().sort_values().reset_index()
    top_lags = top_df.groupby("Lag_Set_Name")["MAPE"].mean().sort_values().reset_index()
    
    # Compute MAPE trends over different forecasting horizons
    mape_trends = top_df.groupby("Model")[mape_columns].mean().reset_index()
    
    return top_models, top_transforms, top_lag_transforms, top_lags, mape_trends

def plot_results(top_models, top_transforms, top_lag_transforms, top_lags, mape_trends):
    plt.figure(figsize=(12, 5))
    sns.barplot(x=top_models["Model"], y=top_models["MAPE"], palette="viridis", hue=top_models["Model"])
    plt.xticks(rotation=45)
    plt.title("Average MAPE per Model")
    plt.show()

    plt.figure(figsize=(12, 5))
    sns.barplot(x=top_transforms["Transforms"], y=top_transforms["MAPE"], palette="coolwarm", hue=top_transforms["Transforms"])
    plt.xticks(rotation=90)
    plt.title("Average MAPE per Transform")
    plt.show()

    plt.figure(figsize=(12, 5))
    sns.barplot(x=top_lag_transforms["Lag Transform Name"], y=top_lag_transforms["MAPE"], palette="Blues", hue=top_lag_transforms["Lag Transform Name"])
    plt.xticks(rotation=90)
    plt.title("Average MAPE per Lag Transform")
    plt.show()

    plt.figure(figsize=(12, 5))
    sns.barplot(x=top_lags["Lag_Set_Name"], y=top_lags["MAPE"], palette="Blues", hue=top_lags["Lag_Set_Name"])
    plt.xticks(rotation=90)
    plt.title("MAPE vs Number of Lags")
    plt.show()
    
    # Plot MAPE trends across different forecasting horizons
    plt.figure(figsize=(12, 5))
    for model in mape_trends["Model"]:
        plt.plot(mape_trends.columns[1:], mape_trends[mape_trends["Model"] == model].values[0][1:], label=model)
    plt.xlabel("Forecasting Horizon (Days)")
    plt.ylabel("MAPE")
    plt.title("MAPE Trends Across Forecast Horizons")
    plt.legend()
    plt.show()

# Example usage:
# top_models, top_transforms, top_lag_transforms, top_lags, mape_trends = analyze_results(df, lag_transforms_map, optimal_lags_map)
# plot_results(top_models, top_transforms, top_lag_transforms, top_lags, mape_trends)


In [4]:
# import pandas as pd
# import numpy as np
# import glob
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Load all results
# results = {}
# for file in glob.glob("results/run_3/*.csv"):
#     dataset_name = file.split("/")[-1].replace(".csv", "")
#     results[dataset_name] = pd.read_csv(file)

# # Combine all datasets into a single DataFrame
# df = pd.concat([df.assign(Dataset=name) for name, df in results.items()], ignore_index=True)

# # Define threshold for MAPE
# mape_threshold = 35  

# # Identify best models per test period
# best_models_per_period = {}
# for test_n in [30, 60, 90, 120, 150, 180, 240, 300, 360, 480, 600, 720, 737]:
#     col = f"test_{test_n}_days"
#     best_models_per_period[test_n] = df[df[col] < mape_threshold].nsmallest(1, col)

# # Calculate variance across all test columns
# test_cols = [col for col in df.columns if "test_" in col]
# df["stability"] = df[test_cols].std(axis=1)

# # Select most stable model
# most_stable_model = df.nsmallest(1, "stability")

In [10]:
import glob
results = {}
for file in glob.glob("results/run_3/*.csv"):
    dataset_name = file.split("\\")[-1].replace(".csv", "")
    results[dataset_name] = pd.read_csv(file)

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1) Define your horizons, threshold, etc.
HORIZONS = [30, 60, 90, 120, 150, 180, 240, 300, 360, 480, 600, 720, 737]
MAPE_COLUMNS = [f"test_{h}_days" for h in HORIZONS]
MAPE_THRESHOLD = 60  # for example

# 2) For each dataframe/dataset:
for dataset_name, df in results.items():
    print("="*80)
    print(f"DATASET: {dataset_name}")
    print("="*80)
    
    # --- (a) Filter out any model whose MAPE above threshold in *any* horizon (or average, your choice) ---
    # For example, filter models that exceed threshold in any horizon:
    mape_columns_local = [c for c in MAPE_COLUMNS if c in df.columns]

    mask_below_threshold = (df[mape_columns_local] <= MAPE_THRESHOLD).all(axis=1)
    df_filtered = df[mask_below_threshold].copy()
    
    # If you wanted to filter by average MAPE across horizons:
    # df['avg_mape'] = df[mape_columns_local].mean(axis=1)
    # df_filtered = df[df['avg_mape'] <= MAPE_THRESHOLD]
    
    # --- (b) Compute stability metrics. Example: standard deviation of MAPE across horizons. ---
    df_filtered['std_mape'] = df_filtered[mape_columns_local].std(axis=1)
    # Also the average MAPE for convenience:
    df_filtered['avg_mape'] = df_filtered[mape_columns_local].mean(axis=1)
    
    # --- (c) Identify best model for each horizon individually ---
    best_models_each_horizon = {}
    if not df_filtered.empty:  # Ensure we have data
        for horizon_col in mape_columns_local:
            if df_filtered[horizon_col].notna().sum() > 0:  # Ensure there's at least one valid value
                idx_min = df_filtered[horizon_col].idxmin()
                best_models_each_horizon[horizon_col] = df_filtered.loc[idx_min]

    
    # Print out best model per horizon
    print("Best Model per Horizon (among models below threshold):")
    for horizon_col, row_data in best_models_each_horizon.items():
        model_name = row_data["Model"]
        transforms = row_data["Transforms"]
        lags = row_data["Lags"]
        value = row_data[horizon_col]
        print(f"  {horizon_col}: {model_name} with MAPE={value:.2f}, transforms={transforms}, lags={lags}")
    
    # --- (d) Identify best stable model overall (lowest std_mape, or some combined criterion) ---
    # For example, pick the row with the lowest std_mape
    if not df_filtered.empty:
        best_stability_idx = df_filtered['std_mape'].idxmin()
        best_stability_row = df_filtered.loc[best_stability_idx]
        print("\nMost Stable Model (lowest std of MAPE across horizons, below threshold):")
        print("  Model:     ", best_stability_row['Model'])
        print("  STD MAPE:  ", f"{best_stability_row['std_mape']:.2f}")
        print("  AVG MAPE:  ", f"{best_stability_row['avg_mape']:.2f}")
    
    # --- (e) Visualization: compare the top N or all models across horizons ---
    # Example: for each model in df_filtered, plot MAPE vs horizon
    # This can give you a sense of stability, slope, etc.
    
    # Convert wide to long format for easier plotting
    df_plot = df_filtered.melt(
        id_vars=["Model", "Transforms", "Lags"],
        value_vars=mape_columns_local,
        var_name="Horizon",
        value_name="MAPE"
    )
    
    # Extract numeric horizon from the text "test_30_days" etc.
    df_plot["Horizon"] = df_plot["Horizon"].str.replace("test_", "").str.replace("_days", "")
    df_plot["Horizon"] = df_plot["Horizon"].astype(int)
    
    # Now you can group by Model and plot:
    models_list = df_plot["Model"].unique()
    
    # plt.figure(figsize=(12, 6))
    # for i, model in enumerate(models_list):
    #     temp = df_plot[df_plot["Model"] == model].sort_values("Horizon")
    #     plt.bar(temp["Horizon"] + i * 10, temp["MAPE"], width=10, label=model)  # Offset bars for visibility

    # plt.title(f"MAPE Comparison Across Horizons - {dataset_name}")
    # plt.xlabel("Horizon (days)")
    # plt.ylabel("MAPE")
    # plt.legend()
    # plt.xticks(df_plot["Horizon"].unique())  # Ensure proper x-axis labels
    # plt.grid(axis='y', linestyle='--', alpha=0.5)
    # plt.show()

    
    # You might also do a bar chart of avg_mape by model, or any other chart that helps you compare.



DATASET: 2_10M_train
Best Model per Horizon (among models below threshold):
DATASET: 2_12M_train
Best Model per Horizon (among models below threshold):
  test_30_days: Lasso with MAPE=18.85, transforms=AutoDifferences(max_diffs=188), lags=[1, 3, 4, 5, 6, 7, 8, 9, 14, 15, 16, 17, 25, 29, 33]
  test_60_days: Ridge with MAPE=21.18, transforms=AutoDifferences(max_diffs=188) | LocalStandardScaler(stats_=[[1.95070557e-02 2.04360268e+01]]), lags=[1, 3, 4, 5, 6, 7, 8, 9, 14, 15, 16, 17, 25, 29, 33]
  test_90_days: Ridge with MAPE=19.84, transforms=AutoDifferences(max_diffs=188) | LocalStandardScaler(stats_=[[1.95070557e-02 2.04360268e+01]]), lags=[1, 3, 4, 5, 6, 7, 8, 9, 14, 15, 16, 17, 25, 29, 33]
  test_120_days: Ridge with MAPE=19.28, transforms=AutoDifferences(max_diffs=188) | LocalStandardScaler(stats_=[[1.95070557e-02 2.04360268e+01]]), lags=[1, 3, 4, 5, 6, 7, 8, 9, 14, 15, 16, 17, 25, 29, 33]
  test_150_days: Ridge with MAPE=18.87, transforms=AutoDifferences(max_diffs=188) | LocalStanda